# Seguro Saúde - Modelagem

### Bibliotecas

In [468]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## modelo
from sklearn import preprocessing
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_validate, cross_val_predict

# check xgboost version
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

/kaggle/input/datasetn/BaseTeste_DataScience.xlsx
/kaggle/input/datasetn/Seguro Sade - Modelagem.xlsx
/kaggle/input/datasetn/Seguro Sade - Teste Final.xlsx


### Funções

In [470]:
def just_check_nan(df):
    # Check NaN Pandas and Numpy
    missing_val_count_by_column = (df.isna().sum())

    columns_with_nan = missing_val_count_by_column[missing_val_count_by_column > 0]
    print('Dados com NaN:')
    print(columns_with_nan)
    print('')
    print('columns_with_nan.shape', columns_with_nan.shape)
    print('')

***
# 1) Definição do problema e objetivo do projeto

Uma administradora de planos de saúde precisa desenvolver um modelo preditivo que estima o custo médio anual gasto por cliente. 

Sua missão será de desenvolver uma modelagem preditiva baseada em algumas informações cadastrais. 

A administradora tem em mãos uma base de teste para posterior avaliação de performance da sua modelagem final.

O resultado de seu estudo será encaminhado para as áreas de negócio para validação funcional.

## 1.1) Dicionario de dados

In [471]:
dicionario = pd.read_excel('/kaggle/input/datasetn/Seguro Sade - Modelagem.xlsx', sheet_name='DICIONÁRIO')
dicionario

,MATRICULA,Matrícula do Cliente
0,IDADE,Idade do Cliente
1,NASCIMENTO,Data de Nascimento do Cliente
2,SEXO,Sexo do Cliente
3,IMC,IMC do Cliente
4,FILHOS,Quantidade de filhos do Cliente
5,FUMANTE,Se o Cliente fuma ou não
6,SIGNO,Signo do Cliente
7,REGIÃO,Região de Moradia do Cliente
8,FACEBOOK,Se o Cliente possui conta do Facebook ou não
9,CLASSE,Segmentação Atual do Cliente


***
# 2) Coleta e preparação dos dados

## 2.1) Coleta dos dados - Modelagem - Representa 90,51% do total

In [472]:
# treinamento
df = pd.read_excel('/kaggle/input/datasetn/Seguro Sade - Modelagem.xlsx', sheet_name='MODELAGEM')
df

,MATRICULA,IDADE,NASCIMENTO,SEXO,IMC,FILHOS,FUMANTE,SIGNO,REGIÃO,FACEBOOK,CLASSE,VALOR
0,207,56.0,1963-12-31,M,39.600,0.0,0.0,Capricórnio,3.0,0.0,7.0,3242.022018
1,852,32.0,1987-10-16,F,37.145,3.0,0.0,Libra,2.0,1.0,2.0,1937.108119
2,463,47.0,1973-01-23,F,24.100,1.0,0.0,Aquário,3.0,1.0,7.0,8023.418951
3,628,41.0,1979-01-08,M,30.780,3.0,1.0,Capricórnio,2.0,1.0,6.0,12109.298838
4,1100,23.0,1996-10-07,M,37.100,3.0,0.0,Libra,3.0,1.0,4.0,1100.182263
...,...,...,...,...,...,...,...,...,...,...,...,...
1206,71,61.0,1958-05-26,F,36.385,1.0,1.0,Gêmeos,2.0,0.0,10.0,14837.175275
1207,1198,20.0,2000-02-11,F,31.920,0.0,0.0,Aquário,4.0,1.0,5.0,691.611254
1208,983,27.0,1992-06-04,F,36.080,0.0,1.0,Gêmeos,1.0,1.0,10.0,11355.932171
1209,299,52.0,1967-05-15,M,32.775,3.0,0.0,Touro,4.0,0.0,1.0,3452.326988


### Checar dados faltantes

In [473]:
just_check_nan(df)

Dados com NaN:
IDADE         5
NASCIMENTO    8
SEXO          7
IMC           5
FILHOS        8
FUMANTE       8
SIGNO         6
REGIÃO        6
FACEBOOK      3
CLASSE        5
dtype: int64

columns_with_nan.shape (10,)



### Apagando dados faltantes

No caso deste problema, como a quantidade de dados faltantes é pouco com relação ao total, optei por deletar os faltantes.

O ideal é averiguar o motivo dos dados faltantes, e se mesmo assim persistir a falta, podemos:

   - Fill nos dados faltantes com:
       - Valores fixos (SimpleImputer)
       - Valores Média, Mediana e Moda (reduz a variação no conjunto de dados)
       - Previsão (KNN)


In [474]:
# df_dropped = df.dropna().reset_index(drop=True)
df_dropped = df

### Checando novamente dados faltantes

In [475]:
just_check_nan(df_dropped)

Dados com NaN:
IDADE         5
NASCIMENTO    8
SEXO          7
IMC           5
FILHOS        8
FUMANTE       8
SIGNO         6
REGIÃO        6
FACEBOOK      3
CLASSE        5
dtype: int64

columns_with_nan.shape (10,)



In [476]:
df_dropped

,MATRICULA,IDADE,NASCIMENTO,SEXO,IMC,FILHOS,FUMANTE,SIGNO,REGIÃO,FACEBOOK,CLASSE,VALOR
0,207,56.0,1963-12-31,M,39.600,0.0,0.0,Capricórnio,3.0,0.0,7.0,3242.022018
1,852,32.0,1987-10-16,F,37.145,3.0,0.0,Libra,2.0,1.0,2.0,1937.108119
2,463,47.0,1973-01-23,F,24.100,1.0,0.0,Aquário,3.0,1.0,7.0,8023.418951
3,628,41.0,1979-01-08,M,30.780,3.0,1.0,Capricórnio,2.0,1.0,6.0,12109.298838
4,1100,23.0,1996-10-07,M,37.100,3.0,0.0,Libra,3.0,1.0,4.0,1100.182263
...,...,...,...,...,...,...,...,...,...,...,...,...
1206,71,61.0,1958-05-26,F,36.385,1.0,1.0,Gêmeos,2.0,0.0,10.0,14837.175275
1207,1198,20.0,2000-02-11,F,31.920,0.0,0.0,Aquário,4.0,1.0,5.0,691.611254
1208,983,27.0,1992-06-04,F,36.080,0.0,1.0,Gêmeos,1.0,1.0,10.0,11355.932171
1209,299,52.0,1967-05-15,M,32.775,3.0,0.0,Touro,4.0,0.0,1.0,3452.326988


### Checagem e mudança de Tipagem

In [477]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211 entries, 0 to 1210
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   MATRICULA   1211 non-null   int64         
 1   IDADE       1206 non-null   float64       
 2   NASCIMENTO  1203 non-null   datetime64[ns]
 3   SEXO        1204 non-null   object        
 4   IMC         1206 non-null   float64       
 5   FILHOS      1203 non-null   float64       
 6   FUMANTE     1203 non-null   float64       
 7   SIGNO       1205 non-null   object        
 8   REGIÃO      1205 non-null   float64       
 9   FACEBOOK    1208 non-null   float64       
 10  CLASSE      1206 non-null   float64       
 11  VALOR       1211 non-null   float64       
dtypes: datetime64[ns](1), float64(8), int64(1), object(2)
memory usage: 113.7+ KB


## 2.2) Preparação dos dados

### Seleção de Features

In [478]:
df_dropped.columns

Index(['MATRICULA', 'IDADE', 'NASCIMENTO', 'SEXO', 'IMC', 'FILHOS', 'FUMANTE',
       'SIGNO', 'REGIÃO', 'FACEBOOK', 'CLASSE', 'VALOR'],
      dtype='object')

### Feature Importance - Criado em outro notebook

In [479]:
# X = df_dropped.drop(['VALOR'], axis=1)

X = df_dropped[['IDADE', 'IMC', 'FILHOS', 'FUMANTE']]

X

,IDADE,IMC,FILHOS,FUMANTE
0,56.0,39.600,0.0,0.0
1,32.0,37.145,3.0,0.0
2,47.0,24.100,1.0,0.0
3,41.0,30.780,3.0,1.0
4,23.0,37.100,3.0,0.0
...,...,...,...,...
1206,61.0,36.385,1.0,1.0
1207,20.0,31.920,0.0,0.0
1208,27.0,36.080,0.0,1.0
1209,52.0,32.775,3.0,0.0


### Seleção do Target

In [480]:
# define target
y = df_dropped['VALOR']

### Variaveis Categoricas

In [481]:
# Criando uma lista de variaveis categoricas
s = (X.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables: ", len(object_cols))
print(object_cols)

Categorical variables:  0
[]


### Variaveis Numéricas

In [482]:
# Criando uma lista de variaveis numéricas
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
print("Numerical variables: ", len(numerical_cols))
print(numerical_cols)

Numerical variables:  4
['IDADE', 'IMC', 'FILHOS', 'FUMANTE']


***
# 3) Modelagem dos dados

## 3.1) Dividindo as features em dados de treino e validação

In [483]:
# Dividindo as features em dados de treino e teste
X_train, X_val, y_train, y_val = train_test_split(X, 
                                                  y, 
                                                  test_size=0.7, 
                                                  random_state = 2048)

## 3.2) Pipeline 

### Pré-processamento para Dados Numéricos

StandardScaler resulta em uma distribuição com desvio padrão igual a 1. A variância também é igual a 1, porque variância = desvio padrão ao quadrado. E 1 ao quadrado = 1.

StandardScaler torna a média da distribuição 0. Cerca de 68% dos valores estarão entre -1 e 1.

Os algoritmos de aprendizado profundo geralmente exigem média zero e variação de unidade. Algoritmos do tipo regressão também se beneficiam de dados normalmente distribuídos com tamanhos de amostra pequenos.

In [484]:
# Pré-processamento para Dados Numéricos
numerical_transformer = Pipeline([
                                  ('scaler', StandardScaler())
                                 ])

### Pré-processamento para Dados Categóricos

In [485]:
# Pré-processamento para Dados Categóricos
categorical_transformer = Pipeline([
                                    ('onehot', OneHotEncoder())
                                   ])

### Pré-processamento de pacote de dados numéricos e categóricos

In [486]:
# Pré-processamento de pacote de dados numéricos e categóricos
preprocessor = ColumnTransformer(transformers=[
                                                ('num', numerical_transformer, numerical_cols),
                                                ('cat', categorical_transformer, object_cols)
                                              ], n_jobs=-1)

### Pipeline

In [487]:
pipeline = Pipeline(steps=[
                            ('preprocessor', preprocessor),
                            ('imputation', KNNImputer()),
                            ('model', GradientBoostingRegressor(random_state=2048))
                          ])

### Fit

In [488]:
# Preprocessing of training data, fit model 
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['IDADE', 'IMC', 'FILHOS',
                                                   'FUMANTE']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  [])])),
                ('imputation', KNNImputer()),
                ('model', GradientBoostingRegressor(random_state=2048))])

### Cross Validation

In [489]:
scores = cross_validate(pipeline, X, y, cv=5, n_jobs=-1,
                        scoring=('r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'))
# scores

In [490]:
print(scores['test_r2'])

[0.90930636 0.86919214 0.81731728 0.78354806 0.80504294]


In [491]:
print(scores['test_neg_mean_squared_error'])

[-1417440.92767161 -1565819.16938661 -2723550.70304608 -2885757.10136092
 -2495212.9511505 ]


In [492]:
print(scores['test_neg_mean_absolute_error'])

[-703.07195037 -765.84595654 -921.28682668 -849.13647176 -843.19353281]


### Previsão

In [493]:
preds = pipeline.predict(X_val)
# preds

***
# 4) Avaliação do modelo

## 4.1) Métricas

### R2 Score - o coeficiente de determinação

Representa a proporção da variância de y. 

A melhor pontuação possível é 1,0 e pode ser negativa

In [494]:
# l1 = [2, 40, 600, 900]
# l2 = [30, 90, 733, 993]
# print('r2', r2_score(l1, l2))
# print('mean_absolute_error', mean_absolute_error(l1, l2))
# print('mean_squared_error', mean_squared_error(l1, l2))

In [495]:
r2_score(y_val, preds)

0.8028829431658848

### Erro médio absoluto (MAE)

Calcula a média dos erros do modelo em valores absolutos, ou seja, a diferença entre o valor real e o previsto, porém é sensível a outliers.

A melhor pontuação possível é 0

In [496]:
mean_absolute_error(y_val, preds)

857.3808122819877

### Erro médio quadrado (MSE)

Calcula a média dos erros do modelo ao quadrado.

A melhor pontuação possível é 0

In [497]:
mean_squared_error(y_val, preds)

2524682.5793801644

## 4.2) Avaliação do modelo com a base de teste - representa 9,49% do total

In [498]:
df_test = pd.read_excel('/kaggle/input/datasetn/Seguro Sade - Teste Final.xlsx', sheet_name='Gabarito')
df_test

,MATRICULA,IDADE,NASCIMENTO,SEXO,IMC,FILHOS,FUMANTE,SIGNO,REGIÃO,FACEBOOK,CLASSE,VALOR
0,1123,22.0,1997-07-27,F,21.280,3.0,0,Leão,4.0,1.0,4,1313.844404
1,743,36.0,1983-08-12,F,25.840,0.0,0,Leão,4.0,1.0,4,1610.509358
2,1151,21.0,1998-06-20,M,28.975,0.0,0,Gêmeos,4.0,1.0,5,582.984174
3,451,47.0,1972-09-22,M,25.460,2.0,0,Virgem,2.0,1.0,4,2821.179327
4,12,64.0,1955-11-17,F,30.115,3.0,0,NaN,4.0,0.0,6,5032.326560
...,...,...,...,...,...,...,...,...,...,...,...,...
122,584,42.0,1977-05-22,M,24.640,0.0,1,Gêmeos,1.0,1.0,5,5968.055535
123,1219,19.0,2000-08-03,M,17.480,0.0,0,Leão,4.0,1.0,8,495.822691
124,857,32.0,1988-01-03,F,23.650,1.0,0,Capricórnio,1.0,1.0,4,5390.287312
125,352,51.0,1969-03-16,F,38.060,0.0,1,Peixes,1.0,0.0,2,13578.105933


### Checar dados faltantes

In [499]:
just_check_nan(df_test)

Dados com NaN:
IDADE       1
SEXO        1
IMC         1
FILHOS      1
SIGNO       1
REGIÃO      1
FACEBOOK    2
dtype: int64

columns_with_nan.shape (7,)



### Apagando dados faltantes

No caso deste problema, como a quantidade de dados faltantes é pouco com relação ao total, optei por deletar os faltantes.

O ideal é averiguar o motivo dos dados faltantes, e se mesmo assim persistir a falta, podemos:

   - Fill nos dados faltantes com:
       - Valores fixos
       - Valores medios
       - Previsão

In [500]:
df_test_dropped = df_test.dropna().reset_index(drop=True)

### Checando novamente dados faltantes

In [501]:
just_check_nan(df_test_dropped)

Dados com NaN:
Series([], dtype: int64)

columns_with_nan.shape (0,)



### Definindo X_test

In [502]:
X_test = df_test_dropped.drop(['VALOR'], axis=1)

X_test

,MATRICULA,IDADE,NASCIMENTO,SEXO,IMC,FILHOS,FUMANTE,SIGNO,REGIÃO,FACEBOOK,CLASSE
0,1123,22.0,1997-07-27,F,21.280,3.0,0,Leão,4.0,1.0,4
1,743,36.0,1983-08-12,F,25.840,0.0,0,Leão,4.0,1.0,4
2,1151,21.0,1998-06-20,M,28.975,0.0,0,Gêmeos,4.0,1.0,5
3,451,47.0,1972-09-22,M,25.460,2.0,0,Virgem,2.0,1.0,4
4,1247,19.0,2001-01-01,F,37.430,0.0,0,Capricórnio,4.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
114,1306,18.0,2001-11-06,M,34.100,0.0,0,Escorpião,1.0,1.0,5
115,584,42.0,1977-05-22,M,24.640,0.0,1,Gêmeos,1.0,1.0,5
116,1219,19.0,2000-08-03,M,17.480,0.0,0,Leão,4.0,1.0,8
117,857,32.0,1988-01-03,F,23.650,1.0,0,Capricórnio,1.0,1.0,4


### Definindo y_test

In [503]:
# define target
y_test = df_test_dropped['VALOR']
y_test

0       1313.844404
1       1610.509358
2        582.984174
3       2821.179327
4        653.844251
           ...     
114      347.709786
115     5968.055535
116      495.822691
117     5390.287312
118    13578.105933
Name: VALOR, Length: 119, dtype: float64

### Previsão

In [504]:
preds_test = pipeline.predict(X_test)

array([ 1241.59352619,  1702.95992824,   961.8708363 ,  4666.92026935,
         982.73942385,  3842.10700168,  4059.57109528,  3085.20986407,
        6091.39339591,  7010.9065058 ,  5850.8940271 ,  4076.64256915,
       11801.3299737 ,  5391.41525102,  3133.32410309,  9640.32481527,
        4789.79809454, 12367.51816955,  5777.94039572,  1857.06585875,
        3396.36506815,  1842.16768287,  4492.11261647,   985.31416047,
        3464.17916137,  7476.35152746,   981.06508307,  1067.88649173,
       14285.9161445 ,  3012.73969856,  2400.45731541,  2498.04399797,
        8144.44247504, 13278.20590961,  3662.04750667,  1774.49583475,
        8413.80369627,  7834.07809   ,  1790.64162314,  3436.30421874,
        1863.18955544,  2106.45790952,  3407.38262499,  4909.52506176,
       11035.3023522 ,  2512.99343061,  4601.72866144,  3311.56698788,
        2098.22312047, 13200.39577381,  5964.4169206 ,   919.62672792,
        4452.29963968,  3455.79863067,  8223.52250987,  4745.49705313,
      

## 4.3) Metricas

### R2 Score - o coeficiente de determinação

Representa a proporção da variância de y. A melhor pontuação possível é 1,0 e pode ser negativa

In [506]:
r2_score(y_test, preds_test)

0.7804335037543312

### Erro médio absoluto (MAE)

Calcula a média dos erros do modelo em valores absolutos, ou seja, a diferença entre o valor real e o previsto, porém é sensível a outliers

In [507]:
mean_absolute_error(y_test, preds_test)

944.944964392613

### Erro médio quadrado (MSE)

Calcula a média dos erros do modelo ao quadrado


In [508]:
mean_squared_error(y_test, preds_test)

2968479.819948533